In [1]:
from enum import Enum
import openai

from keymaker.models import chatgpt, gpt4, LlamaCpp
from keymaker.constraints import RegexConstraint, OptionsConstraint
from keymaker import Prompt

In [2]:
openai.api_key = ""

In [3]:
chat_model=chatgpt()

In [4]:
user_msg = "The weather will be really shitty tomorrow what should I wear?"

In [5]:
prompt_template=f"""
%system%You are an AI assistant that chooses tools for tasks based on a user task and tool descriptions.
Here are the tools you can choose from:
    Tool A: Looks up information on animals
    Tool B: Finds peoples geolocation based on triangulating their phone data
    Tool C: Finds the best outfit to wear today based on the weather
%/system%
%user%{user_msg}
Explain your reasoning in 50 words or less and then choose the appropriate tool.
%/user%
"""

In [6]:
async def print_stream(*args, **kwargs):
    print(args, kwargs)

In [7]:
prompt = Prompt(prompt_template)

In [8]:
prompt = await prompt.complete(model=chat_model, max_tokens=100, stream=print_stream, name='explanation')

(Completion(text = 'Based', start = 505, stop = 510),) {}
(Completion(text = ' on', start = 510, stop = 513),) {}
(Completion(text = ' the', start = 513, stop = 517),) {}
(Completion(text = ' user', start = 517, stop = 522),) {}
(Completion(text = ''s', start = 522, stop = 524),) {}
(Completion(text = ' request', start = 524, stop = 532),) {}
(Completion(text = ' to', start = 532, stop = 535),) {}
(Completion(text = ' find', start = 535, stop = 540),) {}
(Completion(text = ' the', start = 540, stop = 544),) {}
(Completion(text = ' best', start = 544, stop = 549),) {}
(Completion(text = ' outfit', start = 549, stop = 556),) {}
(Completion(text = ' to', start = 556, stop = 559),) {}
(Completion(text = ' wear', start = 559, stop = 564),) {}
(Completion(text = ' tomorrow', start = 564, stop = 573),) {}
(Completion(text = ',', start = 573, stop = 574),) {}
(Completion(text = ' the', start = 574, stop = 578),) {}
(Completion(text = ' appropriate', start = 578, stop = 590),) {}
(Completion(te

In [9]:
prompt.completions.explanation

Completion(text = 'Based on the user's request to find the best outfit to wear tomorrow, the appropriate tool to use would be Tool C, which finds the best outfit based on the weather. This tool will consider the weather conditions and provide recommendations on what to wear to stay comfortable and appropriate for the day's conditions.', start = 505, stop = 822)

In [10]:
llama_model = LlamaCpp(model_path="/Users/nick/Downloads/orca-mini-v2_7b.ggmlv3.q3_K_S.bin")

llama.cpp: loading model from /Users/nick/Downloads/orca-mini-v2_7b.ggmlv3.q3_K_S.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 11 (mostly Q3_K - Small)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 4603.09 MB (+ 1026.00 MB per state)
llama_new_context_with_model: kv self size  =  256.00 MB
AVX = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 1 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 0 | VSX = 0 | 


In [11]:
constraint = OptionsConstraint({'A', 'B', 'C'})

In [12]:
llama_prompt = Prompt(prompt.completions.explanation)+"""
User: Based on this explanation which tool should be chosen?
Assistant:"""

In [13]:
choice_prompt = await llama_prompt.complete(model=llama_model, constraint=constraint)


llama_print_timings:        load time =  7412.85 ms
llama_print_timings:      sample time =     0.71 ms /     1 runs   (    0.71 ms per token,  1408.45 tokens per second)
llama_print_timings: prompt eval time =  7412.77 ms /    84 tokens (   88.25 ms per token,    11.33 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  7417.15 ms


In [14]:
class ToolKinds(Enum):
    A = 'A'
    B = 'B'
    C = 'C'

In [15]:
tool_choice: ToolKinds = ToolKinds(choice_prompt.completions[0])

In [16]:
async def choose_a_tool(prompt, choices_enum: Enum):
    constraint = OptionsConstraint({e.value for e in choices_enum})
    return choices_enum((await prompt.complete(model=llama_model, constraint=constraint)).completions[0])

In [17]:
await choose_a_tool(llama_prompt, ToolKinds)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  7412.85 ms
llama_print_timings:      sample time =     0.69 ms /     1 runs   (    0.69 ms per token,  1453.49 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  1034.10 ms /     1 runs   ( 1034.10 ms per token,     0.97 tokens per second)
llama_print_timings:       total time =  1038.36 ms


<ToolKinds.C: 'C'>